In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.feature_extraction.text import   TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

classes=['toxic','severe_toxic','obscene','threat','insult','identity_hate']
df=pd.read_csv('./Data/train.csv')
df.comment_text=cleaner.clean_text(df['comment_text'])
for category in classes:
    print("There are {} samples for label '{}'".format(len(df.loc[df[category]==1]),category))

Doing translate back...


In [ ]:
df_clean=df.copy()
dfs={}
dfs_test={}
for category in classes:
    df_clean=df_clean.loc[df_clean[category]==0]
    if 1600 > len(df.loc[df[category]==1]) > 1000:
        dfs[category]=(df.loc[df[category]==1]).sample(n=700,axis=0,random_state=4)
        dfs_test[category]=((df.loc[df[category]==1]).drop(dfs[category].index)).sample(n=400,axis=0,random_state=4)
    else:
        dfs_test[category]=(df.loc[df[category]==1]).sample(n=100,axis=0,random_state=4)

In [ ]:
n=6000
m=1200

summary=""
train=df_clean.sample(n=n,axis=0,random_state=4)
df_clean=df_clean.drop(train.index)

for key in dfs:
    train=train.append(dfs[key],ignore_index=True)
train.drop_duplicates(subset ="id", 
                     keep = 'first', inplace = True) 

test=df_clean.sample(n=m,axis=0,random_state=4)
for category in dfs_test:
        test=test.append(dfs_test[category],ignore_index=True)
test.drop_duplicates(subset ="id", 
                     keep = 'first', inplace = True) 



print("Train Data Size: {} ".format(len(train)))
print("Test Data Size: {} ".format(len(test)))

for category in classes:
    print("There are {} samples for label '{}'".format(len(train.loc[train[category]==1]),category))
print("\n")
for category in classes:
    print("There are {} samples for label '{}'".format(len(test.loc[test[category]==1]),category))

In [ ]:
X_train = train.comment_text
X_test = test.comment_text
summary+="Train Samples: {}\nTest Samples: {}\n\n".format(len(X_train),len(X_test))
test.head(3)

In [ ]:
clf_multinomial_bayes = Pipeline([
    ('vect', TfidfVectorizer(stop_words=ENGLISH_STOP_WORDS)),
        ('clf', OneVsRestClassifier(MultinomialNB(fit_prior=True, class_prior=None))),
 ],verbose=False)
clf_decision_tree = Pipeline([
    ('vect', TfidfVectorizer(stop_words=ENGLISH_STOP_WORDS)),
        ('clf', OneVsRestClassifier(DecisionTreeClassifier(max_depth=4))),
 ],verbose=False)

clf_svm = Pipeline([
    ('vect', TfidfVectorizer(stop_words=ENGLISH_STOP_WORDS)),
        ('clf', OneVsRestClassifier(SVC(gamma='auto',kernel='sigmoid'))),
 ],verbose=False)

classifiers={'Multinomial Naive Bayes':clf_multinomial_bayes,'Support Vector Machine':clf_svm,'Decision Tree':clf_decision_tree}

In [ ]:
def get_overall_accuracy(test_df):
    all_corrects=0
    for row in range(len(test_df)):
        current_row=test_df.iloc[row]
        wrong=False
        for label in classes:
            if current_row[label] != current_row['predicted_'+label]:
                wrong=True
                break
        if not wrong:
            all_corrects+=1
    return all_corrects/len(test)

In [ ]:
for each_clf in classifiers:
    print("For classifier {}".format(each_clf))
    copy_test=test.copy()
    overall_acc=0.0
    for each_class in classes:
        clf=classifiers[each_clf]
        clf.fit(X_train,train[each_class])
        predicted = clf.predict(X_test)

        print("There are {} '{}' in test data".format(len(test.loc[test[each_class]==1]),each_class))        
        print("There are {} predicted in test".format(list(predicted).count(1)))
        copy_test['predicted_'+each_class]=predicted
        
        ind_cor_rate=np.mean(predicted == test[each_class])
        overall_acc+=ind_cor_rate
        summary+="Rate of '{}' classifier for class '{}' is {}\n".format(each_clf,each_class,ind_cor_rate)
        print("Accuracy for class '{}' is: {}".format(each_class,ind_cor_rate))
    
    overall_acc/=len(classes)
    summary+='\nOverall class accuracy is {}\n'.format(overall_acc)
    summary+='Overall all class accuracy for classifier {} is {}\n\n'.format(each_clf,get_overall_accuracy(copy_test))

In [ ]:
#save txt
summary_out = open("summary.txt", "w")
summary_out.write(summary)
summary_out.close()